In [5]:
import keras
from keras.layers import LSTM, Dropout, Dense
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle

In [6]:
with open('df_train.pkl', 'rb') as f:
    df_train = pickle.load(f)
    
with open('df_validation.pkl', 'rb') as f:
    df_validation = pickle.load(f)

with open('df_test.pkl', 'rb') as f:
    df_test = pickle.load(f)


In [7]:
df_train_4 = df_train[[
'INTERVAL_IN_FILE',
'BAND_ARR_5',
'BAND_ARR_6',
'BAND_ARR_7',
'BAND_ARR_8',
'BAND_ARR_13',
'BIT_FLAG',
'MAX_VALUE',
'NTC',
'BAND_ARR_2__fft_coefficient__attr_"imag"__coeff_1',
'BAND_ARR_8__fft_coefficient__attr_"real"__coeff_4',
'VALUE'

]].reset_index()

df_validation_4 = df_validation[[
'INTERVAL_IN_FILE',
'BAND_ARR_5',
'BAND_ARR_6',
'BAND_ARR_7',
'BAND_ARR_8',
'BAND_ARR_13',
'BIT_FLAG',
'MAX_VALUE',
'NTC',
'BAND_ARR_2__fft_coefficient__attr_"imag"__coeff_1',
'BAND_ARR_8__fft_coefficient__attr_"real"__coeff_4',
'VALUE'

]].reset_index()

df_test_4 = df_test[[
'INTERVAL_IN_FILE',
'BAND_ARR_5',
'BAND_ARR_6',
'BAND_ARR_7',
'BAND_ARR_8',
'BAND_ARR_13',
'BIT_FLAG',
'MAX_VALUE',
'NTC',
'BAND_ARR_2__fft_coefficient__attr_"imag"__coeff_1',
'BAND_ARR_8__fft_coefficient__attr_"real"__coeff_4',
'VALUE'

]].reset_index()



In [21]:
from sklearn import preprocessing


X_train = df_train_4.loc[:, df_train_4.columns != 'VALUE']

number = preprocessing.LabelEncoder()
X_train['SESSION_INFO_ID'] = number.fit_transform(X_train.SESSION_INFO_ID)       
y_train = df_train_4.VALUE

C:\Users\Arie\AppData\Local\Temp/ipykernel_9968/362572692.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['SESSION_INFO_ID'] = number.fit_transform(X_train.SESSION_INFO_ID)


In [22]:
X_train.shape, y_train.shape

((69445, 12), (69445,))

In [23]:
sequence_length = 10
number_of_features = 12


In [24]:
model = keras.Sequential()
model.add(LSTM(100, input_shape = (sequence_length, number_of_features)))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy"
              , metrics=[keras.metrics.binary_accuracy]
              , optimizer="adam")

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 100)               45200     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 45,301
Trainable params: 45,301
Non-trainable params: 0
_________________________________________________________________


In [25]:
def generate_data(X, y, sequence_length = 10, step = 1):
    X_local = []
    y_local = []
    for start in range(0, len(df_train_4) - sequence_length, step):
        end = start + sequence_length
        X_local.append(X[start:end])
        y_local.append(y[end-1])
    return np.array(X_local), np.array(y_local)

X_sequence, y = generate_data(X_train, y_train)



In [26]:
print(f'Shape: {X_sequence.shape} = (samples, timesteps, features) = (N, T, F)')
print(f'Number of samples, N: {X_sequence.shape[0]}')
print(f'Number of time steps, T: {X_sequence.shape[1]}')
print(f'Number of features, F: {X_sequence.shape[2]}')

Shape: (69435, 10, 12) = (samples, timesteps, features) = (N, T, F)
Number of samples, N: 69435
Number of time steps, T: 10
Number of features, F: 12


In [32]:
X_sequence.shape, y.shape, y_train.shape

((69435, 10, 12), (69435,), (69445,))

In [31]:
model.fit(X_sequence, y, batch_size=64, epochs=4)

Epoch 1/4


TypeError: 'NoneType' object is not callable

In [36]:
X_train.head()

,SESSION_INFO_ID,INTERVAL_IN_FILE,BAND_ARR_5,BAND_ARR_6,BAND_ARR_7,BAND_ARR_8,BAND_ARR_13,BIT_FLAG,MAX_VALUE,NTC,"BAND_ARR_2__fft_coefficient__attr_""imag""__coeff_1","BAND_ARR_8__fft_coefficient__attr_""real""__coeff_4"
0,0,0,0.065059,0.054820,0.055990,0.058916,0.049846,65,0.078078,1393,0.000000,0.000000
1,1,1,0.055083,0.048825,0.044830,0.050023,0.060275,65,0.078783,1418,-0.004584,0.000000
2,1,0,0.073132,0.055827,0.054865,0.063518,0.059672,65,0.295214,1416,-0.004584,0.000000
3,1,2,0.063715,0.055412,0.063122,0.057784,0.056005,65,0.115607,1415,-0.004584,0.000000
4,2,15,5.214829,4.737063,2.248698,1.950095,2.308419,67,5.214829,1509,-775.536100,-136.078768
